In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

from src.data_gen import data_gen
from src.csc_ipca import CSC_IPCA
from src.csc_ife import CSC_IFE
from src.scm import SCM

# set the global font to be Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 8
colors = sns.color_palette()
colors

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

### 1. Finite sample performance of CSC-IPCA estimator

In [2]:
# fix the number of treated units and post treatment periods to 5
N_tr, T1 = 5, 5
# fix the total number of covariates to 9, and the number of factors to 2
L, K = 9, 2 

# fix the drift
drift = 2

In [3]:
# define a function to gen simulated true and estimated att
def simulation_fun(T0, N_co, alpha, n_simulations, method):
    results = {}
    for i in range(n_simulations):
        # generate data
        df = data_gen(T0, T1, N_co, N_tr, L, K, drift)
        # add a constant
        df['const'] = 1
        # compute the true avg ATT
        att = df.query("tr_group==1").groupby('time')['eff'].mean()[-T1:].mean()

        # gen the covariates and observed covariates
        covariates = ['x' + str(i) for i in range(1, L+1)]
        L_obs = int(alpha*len(covariates))
        obs_covariates = list(np.random.choice(covariates, size=L_obs, replace=False)) + ['const']

         # Initialize model based on method
        if method == 'CSC_IPCA':
            model = CSC_IPCA()
        elif method == 'CSC_IFE':
            model = CSC_IFE()
        elif method == 'SCM':
            model = SCM()
        else:
            raise ValueError('method should be one of: CSC_IPCA, CSC_IFE, SCM')
        
        # Fit model
        if method in ['CSC_IPCA', 'CSC_IFE']:
            model.fit(df, 'id', 'time', 'y', 'treated', obs_covariates, K)
            y_syn = model.predict()
            att_est = (df.query("tr_group==1").groupby('time')['y'].mean()[-T1:] - y_syn.mean(axis=0)[-T1:]).mean()
        elif method == 'SCM':
            model.fit(df, 'id', 'time', 'y', 'treated', None)
            y_syn = model.predict()
            att_est = (df.query("tr_group==1").groupby('time')['y'].mean()[-T1:] - y_syn[-T1:]).mean()

        # compute the bias
        bias = att_est - att
        
        results[i] = att, att_est, bias, 
    
    results_df = pd.DataFrame(results, index=['att', 'att_est', 'bias']).T
    avg_bias = results_df.bias.mean()
    sd_att_est = results_df.att_est.std()
    rmse = np.sqrt(((results_df.bias)**2).mean())

    return avg_bias, sd_att_est, rmse

In [4]:
# Define a function to wrap the call to simulation_fun for readability and ease of use with joblib
def simulate(alpha, t, n, method):
    bias, std_att, rmse_att = simulation_fun(T0=t, N_co=n, alpha=alpha, n_simulations=1000, method=method)
    return {"alpha": alpha, "T0": t, "N_co": n, "bias": bias, "std_att": std_att, "rmse_att": rmse_att}

In [5]:
alphas = [1/3, 2/3, 1]
T0 = [10, 20, 40]
N_co = [10, 20, 40]

In [12]:
# Use joblib to run simulations in parallel
# n_jobs=-1 uses all available CPU cores
results1 = Parallel(n_jobs=-1)(delayed(simulate)(alpha, t, n, 'CSC_IPCA') for alpha in alphas for t in T0 for n in N_co)

# Convert the results to a DataFrame
df1 = pd.DataFrame(results1)

In [13]:
# this the 1000 simulations with the constant
df1.pivot_table(index=['T0', 'N_co'], columns='alpha', values=['bias', 'std_att', 'rmse_att']).round(3)

bias                   rmse_att                    std_att  \
alpha   0.333333 0.666667 1.000000 0.333333 0.666667 1.000000 0.333333   
T0 N_co                                                                  
10 10      2.328    0.703    0.130    4.770    3.068    1.642    4.175   
   20      1.367    0.312    0.053    3.484    2.209    0.914    3.260   
   40      1.026    0.196    0.051    2.776    1.752    0.714    2.616   
20 10      2.957    1.029    0.217    4.817    2.696    1.135    3.814   
   20      1.435    0.438    0.055    3.280    1.754    0.745    2.982   
   40      1.093    0.167    0.042    2.613    1.348    0.602    2.430   
40 10      2.905    1.232    0.145    4.911    3.035    0.969    3.972   
   20      1.670    0.399    0.019    3.592    1.718    0.724    3.221   
   40      0.876    0.295    0.006    2.675    1.418    0.574    2.556   

                           
alpha   0.666667 1.000000  
T0 N_co                    
10 10      3.032    1.684  
   20      2.219    1.008  
   40      1.781    0.821  
20 10      2.544    1.179  
   20      1.773    0.860  
   40      1.409    0.757  
40 10      2.797    1.065  
   20      1.737    0.861  
   40      1.441    0.697

In [14]:
# output the results to latex
df1.pivot_table(index=['T0', 'N_co'], columns='alpha', values=['bias', 'std_att', 'rmse_att']).round(3).to_latex('figs/sim_results1.tex', float_format="%.3f")

### 2. Finite sample performance of the CSC-IFE estimator

In [10]:
results2 = Parallel(n_jobs=-1)(delayed(simulate)(alpha, t, n, 'CSC_IFE') for alpha in alphas for t in T0 for n in N_co)

# Convert the results to a DataFrame
df2 = pd.DataFrame(results2)

In [11]:
df2.pivot_table(index=['T0', 'N_co'], columns='alpha', values=['bias', 'std_att', 'rmse_att']).round(3)

bias                   rmse_att                    std_att  \
alpha   0.333333 0.666667 1.000000 0.333333 0.666667 1.000000 0.333333   
T0 N_co                                                                  
10 10      6.478    3.184    0.045    7.996    4.311    0.747    4.701   
   20      6.173    2.885   -0.018    8.070    4.050    0.599    5.245   
   40      4.510    2.516   -0.007    6.785    3.931    0.527    5.096   
20 10      6.650    3.536   -0.007    8.051    4.843    0.777    4.593   
   20      6.402    3.198   -0.013    8.085    4.529    0.587    4.939   
   40      5.690    2.555    0.001    7.633    3.864    0.570    5.111   
40 10      7.353    3.523   -0.008   11.132    5.258    0.696    8.364   
   20      6.904    3.230    0.036    9.185    5.084    0.602    6.053   
   40      5.978    2.928    0.003    9.368    4.850    0.650    7.227   

                           
alpha   0.666667 1.000000  
T0 N_co                    
10 10      2.939    0.854  
   20      2.900    0.769  
   40      3.044    0.675  
20 10      3.336    0.904  
   20      3.272    0.740  
   40      2.935    0.720  
40 10      3.950    0.846  
   20      3.935    0.747  
   40      3.927    0.782

In [15]:
# output the results to latex
df2.pivot_table(index=['T0', 'N_co'], columns='alpha', values=['bias', 'std_att', 'rmse_att']).round(3).to_latex('figs/sim_results2.tex', float_format="%.3f")

### 3. Finite sample performance of the SCM estimator

In [8]:
# n_jobs=-1 uses all available CPU cores
results3 = Parallel(n_jobs=-1)(delayed(simulate)(alpha, t, n, 'SCM') for alpha in alphas for t in T0 for n in N_co)

# Convert the results to a DataFrame
df3 = pd.DataFrame(results3)

In [9]:
df3.pivot_table(index=['T0', 'N_co'], columns='alpha', values=['bias', 'std_att', 'rmse_att']).round(3)

bias                   rmse_att                    std_att  \
alpha   0.333333 0.666667 1.000000 0.333333 0.666667 1.000000 0.333333   
T0 N_co                                                                  
10 10     10.026   10.188    9.909   10.997   11.323   10.964    4.554   
   20      9.874   10.007    9.924   11.011   11.168   11.028    4.872   
   40      9.720   10.088    9.521   10.891   11.388   10.655    4.936   
20 10     10.596   10.526   10.674   12.036   11.841   12.149    5.714   
   20     10.269   10.250   10.113   11.935   11.671   11.745    6.109   
   40      9.719    9.654   10.206   11.309   11.206   12.353    5.810   
40 10     10.678   11.069   11.117   12.794   12.946   13.705    7.087   
   20     10.970   11.207   11.148   13.719   13.459   13.601    8.249   
   40     10.742   10.851   10.221   13.303   13.786   12.684    7.853   

                           
alpha   0.666667 1.000000  
T0 N_co                    
10 10      4.960    4.721  
   20      4.991    4.840  
   40      5.304    4.808  
20 10      5.454    5.816  
   20      5.614    5.985  
   40      5.699    6.986  
40 10      6.731    8.033  
   20      7.451    7.780  
   40      8.520    7.539

In [16]:
# output the results to latex
df3.pivot_table(index=['T0', 'N_co'], columns='alpha', values=['bias', 'std_att', 'rmse_att']).round(3).to_latex('figs/sim_results3.tex', float_format="%.3f")